# Demo: Bench3M for classification Task

In [1]:
from benchmol.trainer.trainer_linear_probing_fusion import (
    FusionModelConfig, OptimConfig,
    create_trainer_from_paths, get_default_task_setup
)

/home/ubuntu/anaconda3/envs/benchmol/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ==== user configs ====
csv_path = "./Bench3M-4M-features-MoleculeNet/bace/bace_processed_ac.csv"
feat_pkl_paths = [
    "./Bench3M-4M-features-MoleculeNet/bace/fingerprints@mcfp4_2048.pkl",
    "./Bench3M-4M-features-MoleculeNet/bace/geometry@unimol_10conf.pkl",
    "./Bench3M-4M-features-MoleculeNet/bace/geometry_image@IEM_3d_10conf.pkl",
]
dataset = "bace"  # used only to choose default eval metric for regression
task_type = "classification"  # or "regression"
split_column = "scaffold_split"  # train/valid/test

device = "cuda:0"  # or "cpu"
runseed = 0
batch_size = 32
lr = 0.005
weights = [0.8, 0.1, 0.1]
num_workers = 2

# Fusion: maps each input to common_dim then fuses with opt
fusion_cfg = FusionModelConfig(
    arch="arch3",
    common_dim=-1,
    hidden_dim_list=None,     # e.g., [512, 256]
    opt="weighted_sum",       # sum/mean/max/min/sub/concat/weighted_sum
    activation="linear",
    weights=weights,          # e.g., [0.2,0.3,0.5] when opt='weighted_sum'
)

optim_cfg = OptimConfig(optim="adam", lr=lr)

In [3]:
# Build trainer (loads CSV+PKLs, builds fusion model + optimizer)
trainer, (idx_train, idx_valid, idx_test) = create_trainer_from_paths(
    csv_path=csv_path,
    feat_pkl_paths=feat_pkl_paths,
    dataset=dataset,
    task_type=task_type,
    split_column=split_column,
    batch_size=batch_size,
    num_workers=num_workers,
    device=device,
    runseed=runseed,
    fusion_cfg=fusion_cfg,
    optim_cfg=optim_cfg,
)

print(trainer.model)

BaseFusionLayer(
  (fc_layers): ModuleList(
    (0): Sequential(
      (linear1): Linear(in_features=2048, out_features=2048, bias=True)
      (activator1): Identity()
    )
    (1): Sequential(
      (linear1): Linear(in_features=512, out_features=2048, bias=True)
      (activator1): Identity()
    )
    (2): Sequential(
      (linear1): Linear(in_features=512, out_features=2048, bias=True)
      (activator1): Identity()
    )
  )
  (activation): Identity()
  (task_layer): Sequential(
    (linear): Linear(in_features=2048, out_features=1, bias=True)
  )
)


In [4]:
# Train
num_epochs = 100
eval_metric, valid_select, min_value, _, _ = get_default_task_setup(dataset, task_type)

results = trainer.train(
    num_epochs=num_epochs,
    eval_metric=eval_metric,
    valid_select=valid_select,
    min_value=min_value,
    dataset=dataset,
    save_dir="./experiments/Bench3M/linear_probing_fusion",
    save_data_prediction_dict=True,
    save_finetune_ckpt=True,
    idx_splits=(idx_train, idx_valid, idx_test),
)
results

[train] dataset: bace; epoch: 0 loss: 0.734: 100%|██████████| 38/38 [00:01<00:00, 22.71it/s]


[train] dataset: bace; epoch: 0 loss: 0.734


[eval on train set] dataset: bace; epoch: 0 loss: 0.603: 100%|██████████| 38/38 [00:00<00:00, 162.19it/s]
[eval on valid set] dataset: bace; epoch: 0 loss: 1.264: 100%|██████████| 5/5 [00:00<00:00, 33.35it/s]
[eval on test set] dataset: bace; epoch: 0 loss: 1.028: 100%|██████████| 5/5 [00:00<00:00, 32.18it/s]


{'dataset': 'bace', 'epoch': 0, 'loss': 0.6029073815596732, 'metric': 'ROCAUC', 'train': 0.8669948630136985, 'valid': 0.7805860805860806, 'test': 0.8113371587549991}
model has been saved as ./experiments/linear_probing_fusion/valid_best.pth


[train] dataset: bace; epoch: 1 loss: 0.443: 100%|██████████| 38/38 [00:00<00:00, 121.53it/s]


...

[train] dataset: bace; epoch: 99 loss: 0.084: 100%|██████████| 38/38 [00:00<00:00, 111.13it/s]


[train] dataset: bace; epoch: 99 loss: 0.084


[eval on train set] dataset: bace; epoch: 99 loss: 0.025: 100%|██████████| 38/38 [00:00<00:00, 151.62it/s]
[eval on valid set] dataset: bace; epoch: 99 loss: 2.240: 100%|██████████| 5/5 [00:00<00:00, 26.00it/s]
[eval on test set] dataset: bace; epoch: 99 loss: 3.392: 100%|██████████| 5/5 [00:00<00:00, 26.21it/s]


{'dataset': 'bace', 'epoch': 99, 'loss': 0.025174719722647416, 'metric': 'ROCAUC', 'train': 0.9996946347031963, 'valid': 0.7146520146520147, 'test': 0.7844722656929229}
final results: {'highest_valid': 0.793040293040293, 'final_train': 0.9423715753424657, 'final_test': 0.8368979307946444, 'highest_train': 0.9996974885844749}


{'highest_valid': 0.793040293040293,
 'final_train': 0.9423715753424657,
 'final_test': 0.8368979307946444,
 'highest_train': 0.9996974885844749,
 'highest_valid_desc': {'accuracy': 0.6622516556291391,
  'ROCAUC': 0.793040293040293,
  'f1': 0.7649769585253456,
  'AUPR': 0.9629934153077016,
  'precision': 0.9540229885057471,
  'recall': 0.6384615384615384,
  'kappa': 0.241056469892579,
  'matthews': 0.31368627729792203,
  'fpr': array([0.        , 0.        , 0.        , 0.04761905, 0.04761905,
         0.0952381 , 0.0952381 , 0.14285714, 0.14285714, 0.19047619,
         0.19047619, 0.23809524, 0.23809524, 0.28571429, 0.28571429,
         0.38095238, 0.38095238, 0.42857143, 0.42857143, 0.47619048,
         0.47619048, 0.52380952, 0.52380952, 0.61904762, 0.61904762,
         0.66666667, 0.66666667, 0.71428571, 0.71428571, 0.76190476,
         0.76190476, 0.85714286, 0.85714286, 0.9047619 , 0.9047619 ,
         0.95238095, 0.95238095, 1.        ]),
  'tpr': array([0.        , 0.00769231, 